In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from tqdm import tqdm

In [2]:
from Main_function import get_Data_On_Page, get_Data_OneProduct_Link 

path = 'chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
#Empty list, to store data from all pages
AllData = []

for page in tqdm(range(26, 31), desc="Crawling pages"):
    main_link = f'https://www.lazada.vn/catalog/?page={page}&q=laptop'
    driver.get(main_link)
    sleep(random.uniform(2,5))                             #Random sleep to mimic human behavior and avoid getting blocked

    df_Laptop = get_Data_On_Page(driver)
    AllData.append(df_Laptop)

Crawling pages: 100%|██████████| 5/5 [02:28<00:00, 29.62s/it]


In [4]:
#Combine into a DataFrame
Data_Laptop_Page26_To_Page30 = pd.concat(AllData, ignore_index=True)
Data_Laptop_Page26_To_Page30.index = np.arange(1, len(Data_Laptop_Page26_To_Page30)+1)
Data_Laptop_Page26_To_Page30

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location
1,laptop,"Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...",https://www.lazada.vn/products/may-tinh-xach-t...,39.000 ₫,17% Off,313 Đã bán,(67),Hồ Chí Minh
2,laptop,Laptop Toshiba Dynabook B35 màn 15.6 inch - Co...,https://www.lazada.vn/products/laptop-toshiba-...,1.727.600 ₫,,None,None,Hà Nội
3,laptop,Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...,https://www.lazada.vn/products/laptop-dell-ins...,3.200.000 ₫,50% Off,None,None,Hà Nội
4,laptop,laptop toshiba z35 cực mạnh - i5 6200U U Ram 1...,https://www.lazada.vn/products/laptop-toshiba-...,3.899 ₫,,14 Đã bán,(0),Quảng Trị
5,laptop,[6.6 SALE TO] MacBook Air 2020 13.3 inches M1-...,https://www.lazada.vn/products/66-sale-to-macb...,17.990.000 ₫,38% Off,3.0K Đã bán,(672),Hồ Chí Minh
...,...,...,...,...,...,...,...,...
196,laptop,Laptop HP Envy x360 2 in 1 14-es0033dx i7-1355...,https://www.lazada.vn/products/laptop-hp-envy-...,17.200.000 ₫,,None,None,Hà Nội
197,laptop,Laptop Asus Vivobook 15 OLED i5-13500H/16GB/51...,https://www.lazada.vn/products/laptop-asus-viv...,19.990.000 ₫,,None,None,Hồ Chí Minh
198,laptop,[Siêu phẩm cao cấp] Laptop HP Spectre X360 Cor...,https://www.lazada.vn/products/sieu-pham-cao-c...,12.990.000 ₫,,None,(1),Nghệ An
199,laptop,LaptopLENOVO THINKPAD X1 Carbon Gen 5 Core i5-...,https://www.lazada.vn/products/laptoplenovo-th...,5.900.000 ₫,14% Off,None,None,Hà Nội


In [5]:
#Initialize dictionary to store details for all products
Products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(Data_Laptop_Page26_To_Page30.Link, desc="Crawling link product"):
    LinkProduct = get_Data_OneProduct_Link(driver, link)
    for key in Products.keys():
        Products[key].append(LinkProduct[key])

Crawling link product: 100%|██████████| 200/200 [1:39:32<00:00, 29.86s/it]


In [6]:
#Add columns to combined_data
for key, value in Products.items():
    Data_Laptop_Page26_To_Page30[key] = value

In [7]:
# Convert to csv
Data_Laptop_Page26_To_Page30.to_csv("Data_Laptop_Page26_To_Page30.csv", index=False)
Data_Laptop_Page26_To_Page30

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Return,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
1,laptop,"Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...",https://www.lazada.vn/products/may-tinh-xach-t...,39.000 ₫,17% Off,313 Đã bán,(67),Hồ Chí Minh,None,None,None,None,None,None,[11],[1],[3],[5],[47]
2,laptop,Laptop Toshiba Dynabook B35 màn 15.6 inch - Co...,https://www.lazada.vn/products/laptop-toshiba-...,1.727.600 ₫,,None,None,Hà Nội,None,None,None,Nhà bán hàng mới,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
3,laptop,Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...,https://www.lazada.vn/products/laptop-dell-ins...,3.200.000 ₫,50% Off,None,None,Hà Nội,6.400.000 ₫,Miễn phí,None,Không đủ thông tin,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
4,laptop,laptop toshiba z35 cực mạnh - i5 6200U U Ram 1...,https://www.lazada.vn/products/laptop-toshiba-...,3.899 ₫,,14 Đã bán,(0),Quảng Trị,None,37.700 ₫,None,97%,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[1]
5,laptop,[6.6 SALE TO] MacBook Air 2020 13.3 inches M1-...,https://www.lazada.vn/products/66-sale-to-macb...,17.990.000 ₫,38% Off,3.0K Đã bán,(672),Hồ Chí Minh,28.990.000 ₫,10.200 ₫,30 Ngày Trả Hàng Miễn Phí,94%,98%,100%,[28],[5],[10],[17],[612]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,laptop,Laptop HP Envy x360 2 in 1 14-es0033dx i7-1355...,https://www.lazada.vn/products/laptop-hp-envy-...,17.200.000 ₫,,None,None,Hà Nội,None,78.950.100 ₫,None,Không đủ thông tin,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
197,laptop,Laptop Asus Vivobook 15 OLED i5-13500H/16GB/51...,https://www.lazada.vn/products/laptop-asus-viv...,19.990.000 ₫,,None,None,Hồ Chí Minh,None,,30 Ngày Trả Hàng Miễn Phí,99%,Không đủ thông tin,100%,[0],[0],[0],[0],[0]
198,laptop,[Siêu phẩm cao cấp] Laptop HP Spectre X360 Cor...,https://www.lazada.vn/products/sieu-pham-cao-c...,12.990.000 ₫,,None,(1),Nghệ An,None,90.400 ₫,None,100%,100%,100%,[0],[0],[0],[0],[1]
199,laptop,LaptopLENOVO THINKPAD X1 Carbon Gen 5 Core i5-...,https://www.lazada.vn/products/laptoplenovo-th...,5.900.000 ₫,14% Off,None,None,Hà Nội,6.900.000 ₫,None,None,97%,100%,100%,[0],[0],[0],[0],[0]
